# Inicialização

In [ ]:
!pip install -r requirements.txt

In [ ]:
import gtfs_kit as gk
import pandas as pd
import swifter
import seaborn as sns

In [ ]:
from pathlib import Path

path = Path("./gtfs_rio-de-janeiro.zip")

feed = gk.read_feed(path, dist_units='km')

feed.validate()

# Parâmetros

In [ ]:
route = '309'
position_parquet_file = './309_2023.parquet'

# Preparação de dados da rota

In [ ]:
route_id = feed.routes[feed.routes['route_short_name'] == route]['route_id'].iloc[0]
route_id

In [ ]:
from shapely import LineString, Point

route_geometry: LineString = feed.geometrize_routes([route_id])['geometry'].iloc[0]
route_geometry


In [ ]:
stop_ids = feed.get_stops(route_ids=[route_id])['stop_id']
geometrized_stops = feed.geometrize_stops(stop_ids)[['stop_id', 'geometry']]
geometrized_stops['projection'] = geometrized_stops['geometry'].apply(lambda x: route_geometry.project(x))
geometrized_stops.sort_values(by='projection', inplace=True)
geometrized_stops.head()

# Preparação de dados de ônibus

In [ ]:
bus_positions = pd.read_parquet(position_parquet_file)
bus_positions.timestamp_gps = pd.to_datetime(bus_positions.timestamp_gps)
bus_positions

# Cálculo de localização

## Localização relativa do ônibus na rota

In [ ]:
def project_location(bus_location: tuple[float, float]):
    bus_location_on_route = route_geometry.interpolate(route_geometry.project(Point(bus_location)))
    return route_geometry.project(bus_location_on_route)

In [ ]:
bus_positions['relative_distance'] = bus_positions[['latitude', 'longitude']].swifter.apply(lambda x: project_location((x['longitude'], x['latitude'])), axis=1)
bus_positions.sample(10)

## Parada onde o ônibus está

In [ ]:
bus_positions['next_stop_index'] = geometrized_stops['projection'].searchsorted(bus_positions['relative_distance'], side='right')
bus_positions['previous_stop_index'] = bus_positions['next_stop_index'] - 1
bus_positions.sample(10)

## Distância do ônibus até a rota

In [ ]:
def distance_from_route(bus_location: tuple[float, float]):
    return route_geometry.distance(Point(bus_location))

In [ ]:
bus_positions['distance_from_route'] = bus_positions[['latitude', 'longitude']].swifter.apply(lambda x: distance_from_route((x['longitude'], x['latitude'])), axis=1)
bus_positions.sample(10)

# Análise

## Validade dos dados - os ônibus estão seguindo a rota?

In [ ]:
sns.violinplot(bus_positions['distance_from_route'])